In [ ]:
!git clone https://github.com/monologg/JointBERT.git

fatal: destination path 'JointBERT' already exists and is not an empty directory.


In [ ]:
def read_data(file_name):
  my_file  = open(file_name,'r')
  data = my_file.read()
  result = data.split("\n")
  my_file.close()
  return result

In [ ]:
train_text = read_data('/content/JointBERT/data/snips/train/seq.in')
train_label = read_data('/content/JointBERT/data/snips/train/label')

In [ ]:
dev_text = read_data('/content/JointBERT/data/snips/dev/seq.in')
dev_label = read_data('/content/JointBERT/data/snips/dev/label')

In [ ]:
test_text = read_data('/content/JointBERT/data/snips/test/seq.in')
test_label = read_data('/content/JointBERT/data/snips/test/label')

In [ ]:
import re
def remove_numbers(lst):
    result = []
    for string in lst:
        result.append(re.sub(r'\d+', '', string))
    return result

In [ ]:
train_text = remove_numbers(train_text)
dev_text = remove_numbers(dev_text)
test_text = remove_numbers(test_text)

In [ ]:
for i in range(len(train_label)):
  if train_label[i]=='':
    print(i)

13084


In [ ]:
train_dict = {
    'text' : train_text[:-1],
    'label' : train_label[:-1]
}
dev_dict = {
    'text' : dev_text[:-1],
    'label' : dev_label[:-1]
}
test_dict = {
    'text' : test_text[:-1],
    'label' : test_label[:-1]
}

In [ ]:
import pandas as pd
train_data = pd.DataFrame(train_dict)
test_data = pd.DataFrame(test_dict)
dev_data = pd.DataFrame(dev_dict)

In [ ]:
train_data.label.value_counts()

PlayMusic               1914
GetWeather              1896
BookRestaurant          1881
RateBook                1876
SearchScreeningEvent    1852
SearchCreativeWork      1847
AddToPlaylist           1818
Name: label, dtype: int64

In [ ]:
test_data.label.value_counts()

AddToPlaylist           124
SearchScreeningEvent    107
SearchCreativeWork      107
GetWeather              104
BookRestaurant           92
PlayMusic                86
RateBook                 80
Name: label, dtype: int64

In [ ]:
dev_data.label.value_counts()

AddToPlaylist           100
BookRestaurant          100
GetWeather              100
PlayMusic               100
RateBook                100
SearchCreativeWork      100
SearchScreeningEvent    100
Name: label, dtype: int64

In [ ]:
cats = list(dev_data.label.unique())
cat_dict = {}
for i in range(len(cats)):
  cat_dict[cats[i]]=i

In [ ]:
train_texts = list(train_data.text)
train_labels = [cat_dict[i] for i in list(train_data.label)]
dev_texts = list(dev_data.text)
dev_labels = [cat_dict[i] for i in list(dev_data.label)]
test_texts = list(test_data.text)
test_labels = [cat_dict[i] for i in list(test_data.label)]

In [ ]:
len(train_texts)

13084

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
import numpy as np

# Define the maximum number of words to keep based on word frequency
MAX_NB_WORDS = 5000

# Define the maximum length of each text sequence
MAX_SEQUENCE_LENGTH = 50

# Define the size of the word embeddings
EMBEDDING_DIM = 100

# Tokenize the text
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_text)

# Convert the text to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
dev_sequences = tokenizer.texts_to_sequences(dev_texts)

# Pad the sequences to have the same length
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
dev_data = pad_sequences(dev_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Define the model architecture
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, np.array(train_labels), validation_data=(dev_data, np.array(dev_labels)), epochs=5, batch_size=64)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data, np.array(test_labels), verbose=2)
print('Test accuracy:', test_acc)


Epoch 1/5
205/205 [==============================] - 38s 161ms/step - loss: 0.6532 - accuracy: 0.8250 - val_loss: 0.0812 - val_accuracy: 0.9800
Epoch 2/5
205/205 [==============================] - 33s 160ms/step - loss: 0.0595 - accuracy: 0.9855 - val_loss: 0.0470 - val_accuracy: 0.9800
Epoch 3/5
205/205 [==============================] - 32s 158ms/step - loss: 0.0325 - accuracy: 0.9913 - val_loss: 0.0676 - val_accuracy: 0.9771
Epoch 4/5
205/205 [==============================] - 35s 168ms/step - loss: 0.0199 - accuracy: 0.9951 - val_loss: 0.0455 - val_accuracy: 0.9829
Epoch 5/5
205/205 [==============================] - 32s 155ms/step - loss: 0.0148 - accuracy: 0.9964 - val_loss: 0.0554 - val_accuracy: 0.9814
22/22 - 0s - loss: 0.0825 - accuracy: 0.9771 - 401ms/epoch - 18ms/step
Test accuracy: 0.977142870426178
